In [ ]:
import os
from dotenv import load_dotenv
import pprint
import requests
import pandas as pd
from requests_oauthlib import OAuth1



In [ ]:

## Get env variables
# Get the current working directory
current_directory = os.getcwd()

# Construct the path to the .env file, which is located in the `scripts` folder four levels up
env_path = os.path.join(
    current_directory,
    '..', '..', '..', '..',
    'scripts',
    '.env'
)


# Load the environment variables from the .env file
load_dotenv(dotenv_path=env_path)

# NS Credentials
NS_REALM_ID = os.getenv('NS_SANDBOX_REALM_ID')
NS_COMPANY_ID = os.getenv('NS_SANDBOX_COMPANY_ID')
NS_CLIENT_KEY = os.getenv('NS_SANDBOX_CLIENT_KEY')
NS_CLIENT_SECRET = os.getenv('NS_SANDBOX_CLIENT_SECRET')
NS_TOKEN_ID = os.getenv('NS_SANDBOX_TOKEN_ID')
NS_TOKEN_SECRET = os.getenv('NS_SANDBOX_TOKEN_SECRET')

# Sanity check
assert all([
    NS_REALM_ID,
    NS_COMPANY_ID,
    NS_CLIENT_KEY,
    NS_CLIENT_SECRET,
    NS_TOKEN_ID,
    NS_TOKEN_SECRET,
]), "Missing one or more NetSuite env vars."
print("NetSuite credentials loaded from environment variables.")


In [ ]:
# NetSuite helper functions
   
def run_suiteql_query(auth, query_string: str):
    """
    Run a paginated SuiteQL query using NetSuite's REST API.

    Args:
        auth: OAuth1 auth object
        query_string (str): A valid SuiteQL query string (without LIMIT/OFFSET)

    Returns:
        list: All matching records
    """

    base_url = f"https://{NS_COMPANY_ID}.suitetalk.api.netsuite.com/services/rest/query/v1/suiteql"
    headers = {
        "Content-Type": "application/json",
        "Prefer": "transient"
    }

    all_items = []
    offset = 0
    limit = 1000

    while True:
        # Use query parameters, not SQL syntax
        url = f"{base_url}?limit={limit}&offset={offset}"
        payload = { "q": query_string }

        response = requests.post(url, json=payload, auth=auth, headers=headers)

        if response.status_code == 200:
            items = response.json().get("items", [])
            all_items.extend(items)

            if len(items) < limit:
                break  # No more records to fetch

            offset += limit
        else:
            print(f"SuiteQL query failed at offset {offset}: {response.status_code}")
            print(response.text)
            break

    return all_items


In [ ]:
# Authenticate to NetSuite
netsuite_auth = OAuth1(
    client_key=NS_CLIENT_KEY,
    client_secret=NS_CLIENT_SECRET,
    resource_owner_key=NS_TOKEN_ID,
    resource_owner_secret=NS_TOKEN_SECRET,
    signature_method='HMAC-SHA256',
    signature_type='AUTH_HEADER',
    realm=NS_REALM_ID
)


## Fetch Netsuite Queries (SuiteQL)



In [ ]:
# Query to get all accounts

# Define the query
query = """
    SELECT
        id, 
        acctnumber,
        fullname
    FROM
        account
"""
# Run the query in NetSuite
records = run_suiteql_query(netsuite_auth, query)

# Print the results
print(f"Fetched {len(records)} account rows")
if records and isinstance(records[0], dict):
    print("Sample keys:", sorted(records[0].keys()))


In [ ]:
# Query to get all departments
query = """
    SELECT
        id, 
        fullname
    FROM
        department
"""
records = run_suiteql_query(netsuite_auth, query)

# Print the results
print(f"Fetched {len(records)} department rows")
if records and isinstance(records[0], dict):
    print("Sample keys:", sorted(records[0].keys()))


In [ ]:
# Query to get all accounting periods

query = "SELECT id, periodname from accountingperiod"
records = run_suiteql_query(netsuite_auth, query)

# Print the results
print(f"Fetched {len(records)} accounting period rows")
if records and isinstance(records[0], dict):
    print("Sample keys:", sorted(records[0].keys()))